In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/football-match-probability-prediction/sample_submission.csv
/kaggle/input/football-match-probability-prediction/train_target_and_scores.csv
/kaggle/input/football-match-probability-prediction/train.csv
/kaggle/input/football-match-probability-prediction/test.csv


In [2]:
#import modules
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

%matplotlib inline

#loading the data
train = pd.read_csv('/kaggle/input/football-match-probability-prediction/train.csv')
test = pd.read_csv('/kaggle/input/football-match-probability-prediction/test.csv')

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3524: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
def is_cup_conversion(df):
    '''convert is_cup column from boolean to 0/1'''
    df.is_cup = df.is_cup.apply(lambda x: np.multiply(x, 1) )
    df['is_cup'].fillna(0, inplace=True) # missing value filled with 0, ie non-cup game
    return df

In [4]:
def has_coach_change(df):
    '''add features indicating whether home and away teams have changed coach'''
    
    df['home_has_coach_change'] = df.apply(lambda r: any([(r['home_team_coach_id']!=r[f'home_team_history_coach_{i}']) 
                                                    & (np.isnan(r[f'home_team_history_coach_{i}'])==False) for i in range(1,11) ]) , axis=1)
    df['home_has_coach_change'] = df['home_has_coach_change'].apply(lambda x: np.multiply(x, 1) )
    df['home_has_coach_change'].fillna(0, inplace=True)
    df['away_has_coach_change'] = df.apply(lambda r: any([(r['away_team_coach_id']!=r[f'away_team_history_coach_{i}']) 
                                                    & (np.isnan(r[f'away_team_history_coach_{i}'])==False) for i in range(1,11) ]) , axis=1)
    df['away_has_coach_change'] = df['away_has_coach_change'].apply(lambda x: np.multiply(x, 1) ).fillna(0, inplace=True)
    df['away_has_coach_change'].fillna(0, inplace=True)
    return df

In [5]:
feature_columns = ['id', 'match_date','league_id','is_cup','home_team_coach_id','away_team_coach_id','home_has_coach_change','away_has_coach_change']
history_columns = [col for col in train.columns if col[-1].isdigit()]

In [6]:
def convert_to_long_form(df, feature_columns, history_columns):
    '''coverting the history columns into long form, for convenience and better speed of manipulation'''
    stubname=['home_team_history_match_date',
           'home_team_history_is_play_home',
           'home_team_history_is_cup',
           'home_team_history_goal',
           'home_team_history_opponent_goal',
           'home_team_history_rating',
           'home_team_history_opponent_rating',
           'home_team_history_coach',
           'home_team_history_league_id',
           'away_team_history_match_date',
           'away_team_history_is_play_home',
           'away_team_history_is_cup',
           'away_team_history_goal',
           'away_team_history_opponent_goal',
           'away_team_history_rating',
           'away_team_history_opponent_rating',
           'away_team_history_coach',
           'away_team_history_league_id']
    df_long=pd.wide_to_long(df[feature_columns+history_columns], stubnames=stubname, i='id', j='match', sep='_')
    df_long=df_long.reset_index().sort_values(by=['id','match'])
    return df_long

In [7]:
def add_match_interval(df):
    '''add features indicating the time gaps between the match days of previous matches for the home and away teams, 
    and how long they played 3 matches and 6 matches,as an reflection of fatique factor'''
    for i in [1,2,5]:
        home_team_history_match_interval = (df.groupby('id').match_date.first().apply(pd.Timestamp) \
             - df.query('match ==1').groupby('id').home_team_history_match_date.first().apply(pd.Timestamp) ) /np.timedelta64(1, "D")
        df=df.join(home_team_history_match_interval.to_frame(name=f'home_team_history_match_interval_{i}'), on='id')
        
        away_team_history_match_interval = (df.groupby('id').match_date.first().apply(pd.Timestamp) \
             - df.query('match ==1').groupby('id').away_team_history_match_date.first().apply(pd.Timestamp) ) /np.timedelta64(1, "D")
        df=df.join(away_team_history_match_interval.to_frame(name=f'away_team_history_match_interval_{i}'), on='id')
        
    return df

In [8]:
def result_point(goal_difference):
    if goal_difference > 0:
        return 3
    elif goal_difference==0:
        return 1
    elif goal_difference < 0:
        return 0
    else:
        return np.NaN

In [9]:
def add_form_features(df):
    '''adding form features for home and away teams, including rating difference and goal difference for each previous match,
    and the average points got, average goal scored, average goal conceded and average goal difference up to that match as previous n-match form'''
    
    df['home_team_history_goal_difference']=df['home_team_history_goal']-df['home_team_history_opponent_goal']
    df['away_team_history_goal_difference']=df['away_team_history_goal']-df['away_team_history_opponent_goal']
    df['home_team_history_point']=df.home_team_history_goal_difference.apply(result_point)
    df['away_team_history_point']=df.away_team_history_goal_difference.apply(result_point)
    df['home_team_history_rating_difference']=df['home_team_history_rating']-df['home_team_history_opponent_rating']
    df['away_team_history_rating_difference']=df['away_team_history_rating']-df['away_team_history_opponent_rating']
    
    home_team_history_goal_info_available=df.groupby('id').home_team_history_goal_difference.count()
    df=df.join(home_team_history_goal_info_available.to_frame(name='home_team_history_goal_info_available'), on='id')
    away_team_history_goal_info_available=df.groupby('id').away_team_history_goal_difference.count()
    df=df.join(away_team_history_goal_info_available.to_frame(name='away_team_history_goal_info_available'), on='id')
    home_team_history_rating_info_available=df.groupby('id').home_team_history_rating_difference.count()
    df=df.join(home_team_history_rating_info_available.to_frame(name='home_team_history_rating_info_available'), on='id')
    away_team_history_rating_info_available=df.groupby('id').away_team_history_rating_difference.count()
    df=df.join(away_team_history_rating_info_available.to_frame(name='away_team_history_rating_info_available'), on='id')
    
        
    for i in [5,10]:
        home_team_history_average_goal_scored=df.query(f'match <= {i}').groupby('id').home_team_history_goal.mean()
        df=df.join(home_team_history_average_goal_scored.to_frame(name=f'home_team_history_average_goal_scored_{i}'), on='id')
        home_team_history_average_goal_conceded=df.query(f'match <= {i}').groupby('id').home_team_history_opponent_goal.mean()
        df=df.join(home_team_history_average_goal_conceded.to_frame(name=f'home_team_history_average_goal_conceded_{i}'), on='id')
        
        away_team_history_average_goal_scored=df.query(f'match <= {i}').groupby('id').away_team_history_goal.mean()
        df=df.join(away_team_history_average_goal_scored.to_frame(name=f'away_team_history_average_goal_scored_{i}'), on='id')
        away_team_history_average_goal_conceded=df.query(f'match <= {i}').groupby('id').away_team_history_opponent_goal.mean()
        df=df.join(away_team_history_average_goal_conceded.to_frame(name=f'away_team_history_average_goal_conceded_{i}'), on='id')
        
        df[f'home_team_history_average_goal_difference_{i}']=df[f'home_team_history_average_goal_scored_{i}']-df[f'home_team_history_average_goal_conceded_{i}']
        df[f'away_team_history_average_goal_difference_{i}']=df[f'away_team_history_average_goal_scored_{i}']-df[f'away_team_history_average_goal_conceded_{i}']
        
        home_team_history_average_point=df.query(f'match <= {i}').groupby('id').home_team_history_point.mean()
        df=df.join(home_team_history_average_point.to_frame(name=f'home_team_history_average_point_{i}'), on='id')
        away_team_history_average_point=df.query(f'match <= {i}').groupby('id').away_team_history_point.mean()
        df=df.join(away_team_history_average_point.to_frame(name=f'away_team_history_average_point_{i}'), on='id')
        home_team_history_average_point_std=df.query(f'match <= {i}').groupby('id').home_team_history_point.std()
        df=df.join(home_team_history_average_point_std.to_frame(name=f'home_team_history_average_point_std_{i}'), on='id')
        away_team_history_average_point_std=df.query(f'match <= {i}').groupby('id').away_team_history_point.std()
        df=df.join(away_team_history_average_point_std.to_frame(name=f'away_team_history_average_point_std_{i}'), on='id')
        
        home_team_history_median_rating_difference=df.query(f'match <= {i}').groupby('id').home_team_history_rating_difference.median()
        df=df.join(home_team_history_median_rating_difference.to_frame(name=f'home_team_history_median_rating_difference_{i}'), on='id')
        away_team_history_median_rating_difference=df.query(f'match <= {i}').groupby('id').away_team_history_rating_difference.median()
        df=df.join(away_team_history_median_rating_difference.to_frame(name=f'away_team_history_median_rating_difference_{i}'), on='id')
        home_team_history_rating_difference_std=df.query(f'match <= {i}').groupby('id').home_team_history_rating_difference.std()
        df=df.join(home_team_history_rating_difference_std.to_frame(name=f'home_team_history_rating_difference_std_{i}'), on='id')
        away_team_history_rating_difference_std=df.query(f'match <= {i}').groupby('id').away_team_history_rating_difference.std()
        df=df.join(away_team_history_rating_difference_std.to_frame(name=f'away_team_history_rating_difference_std_{i}'), on='id')
    
    home_team_home_form_average_goal_scored=df.query('home_team_history_is_play_home==1').groupby('id').home_team_history_goal.mean()
    df=df.join(home_team_home_form_average_goal_scored.to_frame(name='home_team_home_form_average_goal_scored'), on='id')
    home_team_home_form_average_goal_conceded=df.query('home_team_history_is_play_home==1').groupby('id').home_team_history_opponent_goal.mean()
    df=df.join(home_team_home_form_average_goal_conceded.to_frame(name='home_team_home_form_average_goal_conceded'), on='id')
    home_team_home_form_average_goal_difference=df.query('home_team_history_is_play_home==1').groupby('id').home_team_history_goal_difference.mean()
    df=df.join(home_team_home_form_average_goal_difference.to_frame(name='home_team_home_form_average_goal_difference'), on='id')
    home_team_home_form_goal_difference_std=df.query('home_team_history_is_play_home==1').groupby('id').home_team_history_goal_difference.std()
    df=df.join(home_team_home_form_goal_difference_std.to_frame(name='home_team_home_form_goal_difference_std'), on='id')
    home_team_home_form_win_rate=df.query('(home_team_history_is_play_home==1) & (home_team_history_point==3)').groupby('id').home_team_history_point.count()/df.query('home_team_history_is_play_home==1').groupby('id').home_team_history_is_play_home.count()
    df=df.join(home_team_home_form_win_rate.to_frame(name='home_team_home_form_win_rate'), on='id')
    home_team_home_form_draw_rate=df.query('(home_team_history_is_play_home==1) & (home_team_history_point==1)').groupby('id').home_team_history_point.count()/df.query('home_team_history_is_play_home==1').groupby('id').home_team_history_is_play_home.count()
    df=df.join(home_team_home_form_draw_rate.to_frame(name='home_team_home_form_draw_rate'), on='id')
    home_team_home_form_lose_rate=df.query('(home_team_history_is_play_home==1) & (home_team_history_point==0)').groupby('id').home_team_history_point.count()/df.query('home_team_history_is_play_home==1').groupby('id').home_team_history_is_play_home.count()
    df=df.join(home_team_home_form_lose_rate.to_frame(name='home_team_home_form_lose_rate'), on='id')
    home_team_home_form_average_point=df.query('home_team_history_is_play_home==1').groupby('id').home_team_history_point.mean()
    df=df.join(home_team_home_form_average_point.to_frame(name='home_team_home_form_average_point'), on='id')
    home_team_home_form_median_rating_difference=df.query('home_team_history_is_play_home==1').groupby('id').home_team_history_rating_difference.median()
    df=df.join(home_team_home_form_median_rating_difference.to_frame(name='home_team_home_form_median_rating_difference'), on='id')
    home_team_home_form_goal_info_available=df.query('home_team_history_is_play_home==1').groupby('id').home_team_history_goal_difference.count()
    df=df.join(home_team_home_form_goal_info_available.to_frame(name='home_team_home_form_goal_info_available'), on='id')
    home_team_home_form_rating_info_available=df.query('home_team_history_is_play_home==1').groupby('id').home_team_history_rating_difference.count()
    df=df.join(home_team_home_form_rating_info_available.to_frame(name='home_team_home_form_rating_info_available'), on='id')
    
    away_team_away_form_average_goal_scored=df.query('away_team_history_is_play_home==0').groupby('id').away_team_history_goal.mean()
    df=df.join(away_team_away_form_average_goal_scored.to_frame(name='away_team_away_form_average_goal_scored'), on='id')
    away_team_away_form_average_goal_conceded=df.query('away_team_history_is_play_home==0').groupby('id').away_team_history_opponent_goal.mean()
    df=df.join(away_team_away_form_average_goal_conceded.to_frame(name='away_team_away_form_average_goal_conceded'), on='id')
    away_team_away_form_average_goal_difference=df.query('away_team_history_is_play_home==0').groupby('id').away_team_history_goal_difference.mean()
    df=df.join(away_team_away_form_average_goal_difference.to_frame(name='away_team_away_form_average_goal_difference'), on='id')
    away_team_away_form_goal_difference_std=df.query('away_team_history_is_play_home==0').groupby('id').away_team_history_goal_difference.std()
    df=df.join(away_team_away_form_goal_difference_std.to_frame(name='away_team_away_form_goal_difference_std'), on='id')
    away_team_away_form_win_rate=df.query('(away_team_history_is_play_home==0) & (away_team_history_point==3)').groupby('id').away_team_history_point.count()/df.query('away_team_history_is_play_home==0').groupby('id').away_team_history_is_play_home.count()
    df=df.join(away_team_away_form_win_rate.to_frame(name='away_team_away_form_win_rate'), on='id')
    away_team_away_form_draw_rate=df.query('(away_team_history_is_play_home==0) & (away_team_history_point==1)').groupby('id').away_team_history_point.count()/df.query('away_team_history_is_play_home==0').groupby('id').away_team_history_is_play_home.count()
    df=df.join(away_team_away_form_draw_rate.to_frame(name='away_team_away_form_draw_rate'), on='id')
    away_team_away_form_lose_rate=df.query('(away_team_history_is_play_home==0) & (away_team_history_point==0)').groupby('id').away_team_history_point.count()/df.query('away_team_history_is_play_home==0').groupby('id').away_team_history_is_play_home.count()
    df=df.join(away_team_away_form_lose_rate.to_frame(name='away_team_away_form_lose_rate'), on='id')
    away_team_away_form_average_point=df.query('away_team_history_is_play_home==0').groupby('id').away_team_history_point.mean()
    df=df.join(away_team_away_form_average_point.to_frame(name='away_team_away_form_average_point'), on='id')
    away_team_away_form_median_rating_difference=df.query('away_team_history_is_play_home==0').groupby('id').away_team_history_rating_difference.median()
    df=df.join(away_team_away_form_median_rating_difference.to_frame(name='away_team_away_form_median_rating_difference'), on='id')
    away_team_away_form_goal_info_available=df.query('away_team_history_is_play_home==0').groupby('id').away_team_history_goal_difference.count()
    df=df.join(away_team_away_form_goal_info_available.to_frame(name='away_team_away_form_goal_info_available'), on='id')
    away_team_away_form_rating_info_available=df.query('away_team_history_is_play_home==0').groupby('id').away_team_history_rating_difference.count()
    df=df.join(away_team_away_form_rating_info_available.to_frame(name='away_team_away_form_rating_info_available'), on='id')
    
    return df

In [80]:
def add_attack_defense_strength_features(df):
    '''getting goal rates of the league/competition, and calculate the attack and defense strength of a team relative to the league/competition,
    and the expected goals based on this'''
    
    home_history_columns=['home_team_history_match_date',
           'home_team_history_is_play_home',
           'home_team_history_is_cup',
           'home_team_history_goal',
           'home_team_history_opponent_goal',
           'home_team_history_rating',
           'home_team_history_opponent_rating',
           'home_team_history_coach',
           'home_team_history_league_id']
    home_history=df[home_history_columns]
    home_history=home_history.drop_duplicates()
    
    away_history_columns=['away_team_history_match_date',
           'away_team_history_is_play_home',
           'away_team_history_is_cup',
           'away_team_history_goal',
           'away_team_history_opponent_goal',
           'away_team_history_rating',
           'away_team_history_opponent_rating',
           'away_team_history_coach',
           'away_team_history_league_id']
    away_history=df[away_history_columns]
    away_history=away_history.drop_duplicates()
    
    league_total_goal = home_history.groupby('home_team_history_league_id').home_team_history_goal.sum() \
                    + home_history.groupby('home_team_history_league_id').home_team_history_opponent_goal.sum() \
                    + away_history.groupby('away_team_history_league_id').away_team_history_goal.sum() \
                    + away_history.groupby('away_team_history_league_id').away_team_history_opponent_goal.sum()
    league_total_game = home_history.groupby('home_team_history_league_id').home_team_history_goal.count() \
                    + home_history.groupby('home_team_history_league_id').home_team_history_opponent_goal.count() \
                    + away_history.groupby('away_team_history_league_id').away_team_history_goal.count() \
                    + away_history.groupby('away_team_history_league_id').away_team_history_opponent_goal.count() 
    league_average_goal = league_total_goal/league_total_game
    df=df.join(league_average_goal.to_frame(name='league_average_goal'), on='league_id')
    df.league_average_goal=df.league_average_goal.fillna(df.league_average_goal.mean())
    
    league_total_home_goal = home_history.query('home_team_history_is_play_home==1').groupby('home_team_history_league_id').home_team_history_goal.sum() \
                    + home_history.query('home_team_history_is_play_home==0').groupby('home_team_history_league_id').home_team_history_opponent_goal.sum() \
                    + away_history.query('away_team_history_is_play_home==1').groupby('away_team_history_league_id').away_team_history_goal.sum() \
                    + away_history.query('away_team_history_is_play_home==0').groupby('away_team_history_league_id').away_team_history_opponent_goal.sum()
    league_total_home_goal_game = home_history.query('home_team_history_is_play_home==1').groupby('home_team_history_league_id').home_team_history_goal.count() \
                    + home_history.query('home_team_history_is_play_home==0').groupby('home_team_history_league_id').home_team_history_opponent_goal.count() \
                    + away_history.query('away_team_history_is_play_home==1').groupby('away_team_history_league_id').away_team_history_goal.count() \
                    + away_history.query('away_team_history_is_play_home==0').groupby('away_team_history_league_id').away_team_history_opponent_goal.count()
    league_average_home_goal = league_total_home_goal/league_total_home_goal_game
    df=df.join(league_average_home_goal.to_frame(name='league_average_home_goal'), on='league_id')
    df.league_average_home_goal=df.league_average_home_goal.fillna(df.league_average_home_goal.mean())
    
    league_total_away_goal = home_history.query('home_team_history_is_play_home==0').groupby('home_team_history_league_id').home_team_history_goal.sum() \
                    + home_history.query('home_team_history_is_play_home==1').groupby('home_team_history_league_id').home_team_history_opponent_goal.sum() \
                    + away_history.query('away_team_history_is_play_home==0').groupby('away_team_history_league_id').away_team_history_goal.sum() \
                    + away_history.query('away_team_history_is_play_home==1').groupby('away_team_history_league_id').away_team_history_opponent_goal.sum()
    league_total_away_goal_game = home_history.query('home_team_history_is_play_home==0').groupby('home_team_history_league_id').home_team_history_goal.count() \
                    + home_history.query('home_team_history_is_play_home==1').groupby('home_team_history_league_id').home_team_history_opponent_goal.count() \
                    + away_history.query('away_team_history_is_play_home==0').groupby('away_team_history_league_id').away_team_history_goal.count() \
                    + away_history.query('away_team_history_is_play_home==1').groupby('away_team_history_league_id').away_team_history_opponent_goal.count()
    league_average_away_goal = league_total_away_goal/league_total_away_goal_game
    df=df.join(league_average_away_goal.to_frame(name='league_average_away_goal'), on='league_id')
    df.league_average_away_goal=df.league_average_away_goal.fillna(df.league_average_away_goal.mean())
    
    df['home_team_attack_strength'] = df.home_team_history_average_goal_scored_10/df.league_average_goal
    df['home_team_defense_strength'] = df.home_team_history_average_goal_conceded_10/df.league_average_goal
    df['away_team_attack_strength'] = df.away_team_history_average_goal_scored_10/df.league_average_goal
    df['away_team_defense_strength'] = df.away_team_history_average_goal_conceded_10/df.league_average_goal
    df['home_team_expected_goal'] = df.league_average_home_goal*df.home_team_attack_strength*df.away_team_defense_strength
    df['away_team_expected_goal'] = df.league_average_away_goal*df.away_team_attack_strength*df.home_team_defense_strength
    
    
    #getting teams' record of goal scored and conceded in each league
    home_team_league_average_goal_scored=df.groupby(['id','home_team_history_league_id']).home_team_history_goal.agg(['mean']).reset_index()
    home_team_league_average_goal_scored.columns=['id','league_id','home_team_league_average_goal_scored']
    df=pd.merge(df, home_team_league_average_goal_scored, how='left')
    
    home_team_league_average_goal_conceded=df.groupby(['id','home_team_history_league_id']).home_team_history_opponent_goal.agg(['mean']).reset_index()
    home_team_league_average_goal_conceded.columns=['id','league_id','home_team_league_average_goal_conceded']
    df=pd.merge(df, home_team_league_average_goal_conceded, how='left')
    
    away_team_league_average_goal_scored=df.groupby(['id','away_team_history_league_id']).away_team_history_goal.agg(['mean']).reset_index()
    away_team_league_average_goal_scored.columns=['id','league_id','away_team_league_average_goal_scored']
    df=pd.merge(df, away_team_league_average_goal_scored, how='left')
    
    away_team_league_average_goal_conceded=df.groupby(['id','away_team_history_league_id']).away_team_history_opponent_goal.agg(['mean']).reset_index()
    away_team_league_average_goal_conceded.columns=['id','league_id','away_team_league_average_goal_conceded']
    df=pd.merge(df, away_team_league_average_goal_conceded, how='left')
    
    #setting another set of attack, defense and expected goal data based on team's record
    df['home_team_league_attack_strength'] = df.home_team_league_average_goal_scored/df.league_average_goal
    df['home_team_league_defense_strength'] = df.home_team_league_average_goal_conceded/df.league_average_goal
    df['away_team_league_attack_strength'] = df.away_team_league_average_goal_scored/df.league_average_goal
    df['away_team_league_defense_strength'] = df.away_team_league_average_goal_conceded/df.league_average_goal
    df['home_team_league_expected_goal'] = df.league_average_home_goal*df.home_team_league_attack_strength*df.away_team_league_defense_strength
    df['away_team_league_expected_goal'] = df.league_average_away_goal*df.away_team_league_attack_strength*df.home_team_league_defense_strength
    
    #getting coach's record of goal scored and conceded in each league
    home_coach_goal=df.drop_duplicates(subset=home_history_columns)[['home_team_history_coach','home_team_history_league_id', \
                                                             'home_team_history_match_date','home_team_history_goal']].dropna()
    home_coach_goal=home_coach_goal.sort_values(by=['home_team_history_coach','home_team_history_league_id','home_team_history_match_date'],\
                                                ascending=[True,True,True])
    home_coach_goal2=home_coach_goal.set_index(['home_team_history_coach','home_team_history_league_id','home_team_history_match_date']\
                                             ).groupby(level=1, as_index=False).home_team_history_goal.agg(['cumsum','cumcount']).reset_index()
    home_coach_goal2['home_team_coach_league_average_goal_scored']=(home_coach_goal2['cumsum'])/(home_coach_goal2['cumcount']+1)
    home_coach_goal3=home_coach_goal2[['home_team_history_coach','home_team_history_league_id',\
                                       'home_team_history_match_date','home_team_coach_league_average_goal_scored']]
    home_coach_goal3.columns=['home_team_coach_id','league_id','home_team_history_match_date','home_team_coach_league_average_goal_scored']
    home_coach_goal3=home_coach_goal3.drop_duplicates(subset=['home_team_coach_id','league_id','home_team_history_match_date'])
    df=df.merge(home_coach_goal3, how='left')
    
    home_coach_conceded=df.drop_duplicates(subset=home_history_columns)[['home_team_history_coach','home_team_history_league_id', \
                                                             'home_team_history_match_date','home_team_history_opponent_goal']].dropna()
    home_coach_conceded=home_coach_conceded.sort_values(by=['home_team_history_coach','home_team_history_league_id','home_team_history_match_date'],\
                                                ascending=[True,True,True])
    home_coach_conceded2=home_coach_conceded.set_index(['home_team_history_coach','home_team_history_league_id','home_team_history_match_date']\
                                             ).groupby(level=1, as_index=False).home_team_history_opponent_goal.agg(['cumsum','cumcount']).reset_index()
    home_coach_conceded2['home_team_coach_league_average_goal_conceded']=(home_coach_conceded2['cumsum'])/(home_coach_conceded2['cumcount']+1)
    home_coach_conceded3=home_coach_conceded2[['home_team_history_coach','home_team_history_league_id',\
                                       'home_team_history_match_date','home_team_coach_league_average_goal_conceded']]
    home_coach_conceded3.columns=['home_team_coach_id','league_id','home_team_history_match_date','home_team_coach_league_average_goal_conceded']
    home_coach_conceded3=home_coach_conceded3.drop_duplicates(subset=['home_team_coach_id','league_id','home_team_history_match_date'])
    df=df.merge(home_coach_conceded3, how='left')
    
    away_coach_goal=df.drop_duplicates(subset=away_history_columns)[['away_team_history_coach','away_team_history_league_id', \
                                                             'away_team_history_match_date','away_team_history_goal']].dropna()
    away_coach_goal=away_coach_goal.sort_values(by=['away_team_history_coach','away_team_history_league_id','away_team_history_match_date'],\
                                                ascending=[True,True,True])
    away_coach_goal2=away_coach_goal.set_index(['away_team_history_coach','away_team_history_league_id','away_team_history_match_date']\
                                             ).groupby(level=1, as_index=False).away_team_history_goal.agg(['cumsum','cumcount']).reset_index()
    away_coach_goal2['away_team_coach_league_average_goal_scored']=(away_coach_goal2['cumsum'])/(away_coach_goal2['cumcount']+1)
    away_coach_goal3=away_coach_goal2[['away_team_history_coach','away_team_history_league_id',\
                                       'away_team_history_match_date','away_team_coach_league_average_goal_scored']]
    away_coach_goal3.columns=['away_team_coach_id','league_id','away_team_history_match_date','away_team_coach_league_average_goal_scored']
    away_coach_goal3=away_coach_goal3.drop_duplicates(subset=['away_team_coach_id','league_id','away_team_history_match_date'])
    df=df.merge(away_coach_goal3, how='left')
    
    away_coach_conceded=df.drop_duplicates(subset=away_history_columns)[['away_team_history_coach','away_team_history_league_id', \
                                                             'away_team_history_match_date','away_team_history_opponent_goal']].dropna()
    away_coach_conceded=away_coach_conceded.sort_values(by=['away_team_history_coach','away_team_history_league_id','away_team_history_match_date'],\
                                                ascending=[True,True,True])
    away_coach_conceded2=away_coach_conceded.set_index(['away_team_history_coach','away_team_history_league_id','away_team_history_match_date']\
                                             ).groupby(level=1, as_index=False).away_team_history_opponent_goal.agg(['cumsum','cumcount']).reset_index()
    away_coach_conceded2['away_team_coach_league_average_goal_conceded']=(away_coach_conceded2['cumsum'])/(away_coach_conceded2['cumcount']+1)
    away_coach_conceded3=away_coach_conceded2[['away_team_history_coach','away_team_history_league_id',\
                                       'away_team_history_match_date','away_team_coach_league_average_goal_conceded']]
    away_coach_conceded3.columns=['away_team_coach_id','league_id','away_team_history_match_date','away_team_coach_league_average_goal_conceded']
    away_coach_conceded3=away_coach_conceded3.drop_duplicates(subset=['away_team_coach_id','league_id','away_team_history_match_date'])
    df=df.merge(away_coach_conceded3, how='left')
    
    #setting another set of attack, defense and expected goal data based on coach's record
    df['home_team_coach_attack_strength'] = df.home_team_coach_league_average_goal_scored/df.league_average_goal
    df['home_team_coach_defense_strength'] = df.home_team_coach_league_average_goal_conceded/df.league_average_goal
    df['away_team_coach_attack_strength'] = df.away_team_coach_league_average_goal_scored/df.league_average_goal
    df['away_team_coach_defense_strength'] = df.away_team_coach_league_average_goal_conceded/df.league_average_goal
    df['home_team_coach_expected_goal'] = df.league_average_home_goal*df.home_team_coach_attack_strength*df.away_team_coach_defense_strength
    df['away_team_coach_expected_goal'] = df.league_average_away_goal*df.away_team_coach_attack_strength*df.home_team_coach_defense_strength
    
    return df

In [64]:
train2=is_cup_conversion(train)
train2=has_coach_change(train2)
train2=convert_to_long_form(train2, feature_columns, history_columns)
train2=add_match_interval(train2)
train2=add_form_features(train2)

In [12]:
home_history_columns=['home_team_history_match_date',
           'home_team_history_is_play_home',
           'home_team_history_is_cup',
           'home_team_history_goal',
           'home_team_history_opponent_goal',
           'home_team_history_rating',
           'home_team_history_opponent_rating',
           'home_team_history_coach',
           'home_team_history_league_id']

In [13]:
train2

,id,match,home_has_coach_change,away_has_coach_change,home_team_coach_id,is_cup,away_team_coach_id,match_date,league_id,home_team_history_match_date,...,away_team_away_form_average_goal_conceded,away_team_away_form_average_goal_difference,away_team_away_form_goal_difference_std,away_team_away_form_win_rate,away_team_away_form_draw_rate,away_team_away_form_lose_rate,away_team_away_form_average_point,away_team_away_form_median_rating_difference,away_team_away_form_goal_info_available,away_team_away_form_rating_info_available
913,11428294,1,1,0,468693.0,0.0,257433.0,2019-12-03 23:00:00,755,2019-11-27 21:00:00,...,1.4,-1.2,0.836660,NaN,0.2,0.8,0.2,-2.846002,5.0,5.0
111851,11428294,2,1,0,468693.0,0.0,257433.0,2019-12-03 23:00:00,755,2019-11-23 23:00:00,...,1.4,-1.2,0.836660,NaN,0.2,0.8,0.2,-2.846002,5.0,5.0
222789,11428294,3,1,0,468693.0,0.0,257433.0,2019-12-03 23:00:00,755,2019-11-12 21:00:00,...,1.4,-1.2,0.836660,NaN,0.2,0.8,0.2,-2.846002,5.0,5.0
333727,11428294,4,1,0,468693.0,0.0,257433.0,2019-12-03 23:00:00,755,2019-11-03 21:30:00,...,1.4,-1.2,0.836660,NaN,0.2,0.8,0.2,-2.846002,5.0,5.0
444665,11428294,5,1,0,468693.0,0.0,257433.0,2019-12-03 23:00:00,755,2019-10-25 21:00:00,...,1.4,-1.2,0.836660,NaN,0.2,0.8,0.2,-2.846002,5.0,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
665423,18053837,6,0,0,NaN,0.0,NaN,2021-04-30 13:00:00,1642,2021-03-19 13:00:00,...,1.4,0.4,1.140175,0.4,0.4,0.2,1.6,-1.740500,5.0,5.0
776361,18053837,7,0,0,NaN,0.0,NaN,2021-04-30 13:00:00,1642,2021-03-12 12:00:00,...,1.4,0.4,1.140175,0.4,0.4,0.2,1.6,-1.740500,5.0,5.0
887299,18053837,8,0,0,NaN,0.0,NaN,2021-04-30 13:00:00,1642,2020-03-13 12:00:00,...,1.4,0.4,1.140175,0.4,0.4,0.2,1.6,-1.740500,5.0,5.0
998237,18053837,9,0,0,NaN,0.0,NaN,2021-04-30 13:00:00,1642,2020-03-06 12:00:00,...,1.4,0.4,1.140175,0.4,0.4,0.2,1.6,-1.740500,5.0,5.0


In [65]:
xxx=train2.drop_duplicates(subset=home_history_columns)[['home_team_history_coach','home_team_history_league_id','home_team_history_match_date','home_team_history_goal']]

In [16]:
xxx.shape

(263997, 5)

In [53]:
xxx.head(50)

,id,home_team_history_coach,home_team_history_league_id,home_team_history_match_date,home_team_history_goal
913,11428294,468693.0,755.0,2019-11-27 21:00:00,0.0
111851,11428294,468693.0,755.0,2019-11-23 23:00:00,1.0
222789,11428294,468693.0,755.0,2019-11-12 21:00:00,1.0
333727,11428294,468693.0,755.0,2019-11-03 21:30:00,1.0
444665,11428294,468693.0,755.0,2019-10-25 21:00:00,2.0
555603,11428294,468693.0,755.0,2019-10-20 22:45:00,1.0
666541,11428294,235734.0,1725.0,2019-10-10 23:00:00,0.0
777479,11428294,235734.0,755.0,2019-10-07 23:00:00,0.0
888417,11428294,235734.0,755.0,2019-09-30 22:00:00,0.0
999355,11428294,235734.0,755.0,2019-09-26 22:00:00,0.0


In [67]:
xxx=xxx.dropna()

In [18]:
xxx.home_team_history_coach.value_counts()

-999.0         48095
 37252510.0    14669
 524307.0        104
 2511092.0       102
 459064.0         99
               ...  
 4599443.0         1
 1552359.0         1
 37442631.0        1
 457315.0          1
 37528942.0        1
Name: home_team_history_coach, Length: 9942, dtype: int64

In [19]:
xxy=xxx.loc[xxx.home_team_history_coach==2511092].sort_values(by=['home_team_history_league_id','home_team_history_match_date'], ascending=[True,True])

In [20]:
xxz=xxy.set_index(['home_team_history_league_id','home_team_history_match_date']).groupby(level=0, as_index=False).home_team_history_goal.agg(['cumsum','cumcount'])

In [21]:
xxz['home_team_coach_league_mean_goal']=(xxz['cumsum'])/(xxz['cumcount']+1)
xxz

cumsum  cumcount  \
home_team_history_league_id home_team_history_match_date                     
2.0                         2020-08-25 18:00:00              3.0         0   
                            2020-09-15 18:00:00              5.0         1   
                            2020-09-22 19:00:00              6.0         2   
                            2020-09-30 19:00:00              7.0         3   
5.0                         2020-10-22 16:55:00              1.0         0   
...                                                          ...       ...   
1313.0                      2021-04-18 23:00:00              8.0         5   
                            2021-04-23 23:00:00             10.0         6   
                            2021-04-25 23:00:00             11.0         7   
1483.0                      2021-04-08 00:30:00              2.0         0   
                            2021-04-15 00:30:00              3.0         1   

                                                          home_team_coach_league_mean_goal  
home_team_history_league_id home_team_history_match_date                                    
2.0                         2020-08-25 18:00:00                                   3.000000  
                            2020-09-15 18:00:00                                   2.500000  
                            2020-09-22 19:00:00                                   2.000000  
                            2020-09-30 19:00:00                                   1.750000  
5.0                         2020-10-22 16:55:00                                   1.000000  
...                                                                                    ...  
1313.0                      2021-04-18 23:00:00                                   1.333333  
                            2021-04-23 23:00:00                                   1.428571  
                            2021-04-25 23:00:00                                   1.375000  
1483.0                      2021-04-08 00:30:00                                   2.000000  
                            2021-04-15 00:30:00                                   1.500000  

[102 rows x 3 columns]

In [22]:
xxz.reset_index().head(50)

,home_team_history_league_id,home_team_history_match_date,cumsum,cumcount,home_team_coach_league_mean_goal
0,2.0,2020-08-25 18:00:00,3.0,0,3.000000
1,2.0,2020-09-15 18:00:00,5.0,1,2.500000
2,2.0,2020-09-22 19:00:00,6.0,2,2.000000
3,2.0,2020-09-30 19:00:00,7.0,3,1.750000
4,5.0,2020-10-22 16:55:00,1.0,0,1.000000
5,5.0,2020-10-29 20:00:00,1.0,1,0.500000
6,5.0,2020-11-05 17:55:00,5.0,2,1.666667
7,325.0,2019-09-14 16:00:00,3.0,0,3.000000
8,325.0,2019-09-22 15:30:00,5.0,1,2.500000
9,325.0,2019-09-29 16:30:00,7.0,2,2.333333


In [68]:
xxx.sort_values(by=['home_team_history_coach','home_team_history_league_id','home_team_history_match_date'], ascending=[True,True,True], inplace=True)

In [69]:
xxx.set_index(['home_team_history_coach','home_team_history_league_id','home_team_history_match_date']).groupby(level=1, as_index=False).home_team_history_goal.agg(['cumsum','cumcount']).reset_index()

,home_team_history_coach,home_team_history_league_id,home_team_history_match_date,cumsum,cumcount
0,2.0,1007.0,2020-11-27 14:00:00,0.0,0
1,2.0,1007.0,2020-12-01 14:00:00,0.0,1
2,2.0,1007.0,2020-12-05 14:00:00,0.0,2
3,2.0,1007.0,2020-12-10 14:00:00,0.0,3
4,2.0,1007.0,2020-12-15 14:00:00,2.0,4
...,...,...,...,...,...
215897,37566929.0,1879.0,2021-04-25 12:00:00,58.0,32
215898,37566931.0,1554.0,2021-04-17 11:00:00,264.0,178
215899,37567242.0,14.0,2021-04-20 18:00:00,1951.0,1615
215900,37567242.0,14.0,2021-04-24 14:00:00,1951.0,1616


In [70]:
xxx2=xxx.set_index(['home_team_history_coach','home_team_history_league_id','home_team_history_match_date']).groupby(level=1, as_index=False).home_team_history_goal.agg(['cumsum','cumcount']).reset_index()

In [71]:
xxx2['home_team_coach_league_mean_goal']=(xxx2['cumsum'])/(xxx2['cumcount']+1)

In [72]:
xxx3=xxx2[['home_team_history_coach','home_team_history_league_id','home_team_history_match_date','home_team_coach_league_mean_goal']]
xxx3.columns=['home_team_coach_id','league_id','home_team_history_match_date','home_team_coach_league_mean_goal']

In [60]:
xxx3.head(50)

,home_team_coach_id,league_id,home_team_history_match_date,home_team_coach_league_mean_goal
0,2.0,1007.0,2020-11-27 14:00:00,0.000000
1,2.0,1007.0,2020-12-01 14:00:00,0.000000
2,2.0,1007.0,2020-12-05 14:00:00,0.000000
3,2.0,1007.0,2020-12-10 14:00:00,0.000000
4,2.0,1007.0,2020-12-15 14:00:00,0.400000
5,2.0,1007.0,2020-12-20 14:00:00,0.500000
6,2.0,1007.0,2020-12-26 14:00:00,0.714286
7,2.0,1007.0,2021-01-03 11:30:00,1.000000
8,2.0,1007.0,2021-01-06 14:00:00,1.000000
9,2.0,1007.0,2021-01-09 14:00:00,1.000000


In [73]:
xxx3[['home_team_coach_id','league_id','home_team_history_match_date']].duplicated().value_counts()

False    206230
True       9672
dtype: int64

In [76]:
xxx3=xxx3.drop_duplicates(subset=['home_team_coach_id','league_id','home_team_history_match_date'])

In [77]:
yyy=train2.merge(xxx3, how='left', validate='many_to_one')

In [78]:
yyy

,id,match,home_has_coach_change,away_has_coach_change,home_team_coach_id,is_cup,away_team_coach_id,match_date,league_id,home_team_history_match_date,...,away_team_away_form_average_goal_difference,away_team_away_form_goal_difference_std,away_team_away_form_win_rate,away_team_away_form_draw_rate,away_team_away_form_lose_rate,away_team_away_form_average_point,away_team_away_form_median_rating_difference,away_team_away_form_goal_info_available,away_team_away_form_rating_info_available,home_team_coach_league_mean_goal
0,11428294,1,1,0,468693.0,0.0,257433.0,2019-12-03 23:00:00,755,2019-11-27 21:00:00,...,-1.2,0.836660,NaN,0.2,0.8,0.2,-2.846002,5.0,5.0,1.203846
1,11428294,2,1,0,468693.0,0.0,257433.0,2019-12-03 23:00:00,755,2019-11-23 23:00:00,...,-1.2,0.836660,NaN,0.2,0.8,0.2,-2.846002,5.0,5.0,1.208494
2,11428294,3,1,0,468693.0,0.0,257433.0,2019-12-03 23:00:00,755,2019-11-12 21:00:00,...,-1.2,0.836660,NaN,0.2,0.8,0.2,-2.846002,5.0,5.0,1.209302
3,11428294,4,1,0,468693.0,0.0,257433.0,2019-12-03 23:00:00,755,2019-11-03 21:30:00,...,-1.2,0.836660,NaN,0.2,0.8,0.2,-2.846002,5.0,5.0,1.210117
4,11428294,5,1,0,468693.0,0.0,257433.0,2019-12-03 23:00:00,755,2019-10-25 21:00:00,...,-1.2,0.836660,NaN,0.2,0.8,0.2,-2.846002,5.0,5.0,1.210938
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1109375,18053837,6,0,0,NaN,0.0,NaN,2021-04-30 13:00:00,1642,2021-03-19 13:00:00,...,0.4,1.140175,0.4,0.4,0.2,1.6,-1.740500,5.0,5.0,NaN
1109376,18053837,7,0,0,NaN,0.0,NaN,2021-04-30 13:00:00,1642,2021-03-12 12:00:00,...,0.4,1.140175,0.4,0.4,0.2,1.6,-1.740500,5.0,5.0,NaN
1109377,18053837,8,0,0,NaN,0.0,NaN,2021-04-30 13:00:00,1642,2020-03-13 12:00:00,...,0.4,1.140175,0.4,0.4,0.2,1.6,-1.740500,5.0,5.0,NaN
1109378,18053837,9,0,0,NaN,0.0,NaN,2021-04-30 13:00:00,1642,2020-03-06 12:00:00,...,0.4,1.140175,0.4,0.4,0.2,1.6,-1.740500,5.0,5.0,NaN


In [79]:
yyy.loc[yyy.home_team_history_coach==2511092].sample(30)

,id,match,home_has_coach_change,away_has_coach_change,home_team_coach_id,is_cup,away_team_coach_id,match_date,league_id,home_team_history_match_date,...,away_team_away_form_average_goal_difference,away_team_away_form_goal_difference_std,away_team_away_form_win_rate,away_team_away_form_draw_rate,away_team_away_form_lose_rate,away_team_away_form_average_point,away_team_away_form_median_rating_difference,away_team_away_form_goal_info_available,away_team_away_form_rating_info_available,home_team_coach_league_mean_goal
773792,17175673,3,0,0,2511092.0,0.0,1553099.0,2020-09-18 16:00:00,325,2020-08-25 18:00:00,...,0.50,1.000000,0.750000,NaN,0.250000,2.250000,-0.860005,4.0,4.0,NaN
54579,11890481,10,0,0,2511092.0,0.0,1467991.0,2020-01-19 16:00:00,325,2019-11-24 16:00:00,...,-1.00,0.000000,NaN,NaN,1.000000,0.000000,-4.138272,3.0,3.0,1.203735
54709,11890494,10,0,0,2511092.0,0.0,30841489.0,2020-01-27 16:00:00,325,2019-12-07 16:00:00,...,-0.25,2.629956,0.500000,0.250000,0.250000,1.750000,-4.874926,4.0,4.0,1.204738
54984,11890522,5,0,0,2511092.0,0.0,456213.0,2020-02-23 17:30:00,325,2020-02-02 17:30:00,...,0.50,0.577350,0.500000,0.500000,NaN,2.000000,5.079286,4.0,4.0,1.216667
54142,11890438,3,0,0,2511092.0,0.0,108536.0,2019-12-07 16:00:00,325,2019-11-09 17:30:00,...,-0.75,1.258306,0.250000,NaN,0.750000,0.750000,-2.024218,4.0,4.0,1.204082
370047,12025093,8,0,0,2511092.0,0.0,456040.0,2021-01-18 22:00:00,648,2020-12-20 00:00:00,...,0.60,0.894427,0.400000,0.600000,NaN,1.800000,-0.258382,5.0,5.0,1.265105
1028884,17780012,5,0,0,2511092.0,1.0,453012.0,2021-04-15 00:30:00,1483,2021-03-07 21:00:00,...,-0.25,1.707825,0.250000,0.250000,0.500000,1.000000,0.743843,4.0,4.0,NaN
875100,17323696,1,0,0,2511092.0,0.0,37253382.0,2020-10-04 15:15:00,325,2020-09-30 19:00:00,...,-0.80,1.095445,0.200000,NaN,0.800000,0.600000,-3.759213,5.0,5.0,NaN
408908,13226046,9,0,0,2511092.0,0.0,37253382.0,2020-06-20 17:30:00,325,2020-02-05 19:30:00,...,-1.60,2.073644,0.200000,0.200000,0.600000,0.800000,-4.872300,5.0,5.0,NaN
453432,16454699,3,0,0,2511092.0,0.0,456085.0,2020-07-19 17:00:00,325,2020-07-08 18:00:00,...,-0.75,1.500000,0.250000,0.250000,0.500000,1.000000,-5.293275,4.0,4.0,1.217676


In [ ]:
def reshape(df):
    '''reshape the dataframe back to wide form'''
    history_columns=['home_team_history_is_play_home',
              'home_team_history_is_cup',
              'home_team_history_goal',
              'home_team_history_opponent_goal',
              'home_team_history_rating',
              'home_team_history_opponent_rating',
              'home_team_history_coach',
              'home_team_history_league_id',
              'home_team_history_goal_difference',
              'home_team_history_point',
              'home_team_history_rating_difference',
              'away_team_history_is_play_home',
              'away_team_history_is_cup',
              'away_team_history_goal',
              'away_team_history_opponent_goal',
              'away_team_history_rating',
              'away_team_history_opponent_rating',
              'away_team_history_coach',
              'away_team_history_league_id',
              'away_team_history_goal_difference',
              'away_team_history_point',
              'away_team_history_rating_difference']
    drop_columns=['match','match_date','home_team_history_match_date','away_team_history_match_date']
    df_history=df[history_columns+['id','match']].set_index(['id','match']).unstack(level=1)
    df_history.columns = df_history.columns.map('{0[0]}_{0[1]}'.format) 
    df_history=df_history.reset_index()
    df_new=df.groupby('id').first().reset_index().drop(history_columns+drop_columns, axis=1)
    df_new=df_new.merge(df_history, how='left')
    
    return df_new
    

In [ ]:
def preprocession(df):
    df=is_cup_conversion(df)
    df=has_coach_change(df)
    df=convert_to_long_form(df, feature_columns, history_columns)
    df=add_match_interval(df)
    df=add_form_features(df)
    df=add_attack_defense_strength_features(df)
    df.replace([np.inf, -np.inf], np.nan, inplace=True)
    #df=df.groupby('id').first().reset_index()
    df=reshape(df)

    return df

In [ ]:
train_p=preprocession(train)

In [ ]:
for i in train_p.columns:
    print(i, train_p.loc[467,i])

In [ ]:
train_p

In [ ]:
X=train_p.drop('id', axis=1)
y=train.sort_values(by='id')['target']

In [ ]:
X

In [ ]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
label_encoder = label_encoder.fit(y)
label_encoded_y = label_encoder.transform(y)

In [ ]:
from sklearn import model_selection
X_train, X_valid, y_train, y_valid = model_selection.train_test_split(X, label_encoded_y, train_size=0.8, random_state=42)

In [ ]:
import xgboost as xgb
d_train = xgb.DMatrix(X_train, y_train)
d_test = xgb.DMatrix(X_valid, y_valid)

In [ ]:
params_1 = {
    'booster': 'gbtree',
    'max_depth': 5, 
    'learning_rate': 0.025,
    'min_split_loss': 0.01,
    'min_child_weight': 5,
    'subsample': 0.8,
    'colsample_bytree': 0.25,
    'alpha': 2,
    'lambda': 3,
    'objective': 'multi:softprob',
    'num_class': 3,
    'eval_metric':'mlogloss'
    }

num_round = 1000
evallist = [(d_train, 'train'), (d_test, 'eval')]

In [ ]:
bst = xgb.train(params_1, d_train, num_round, evallist, early_stopping_rounds=20)

In [ ]:
xgb.plot_importance(bst, max_num_features=200)
fig = plt.gcf()
fig.set_size_inches(18.5, 30.5)

In [ ]:
xgb.plot_tree(bst, num_trees=1)
fig = plt.gcf()
fig.set_size_inches(160, 100)

In [ ]:
bst.best_iteration

In [ ]:
bst.best_score

In [ ]:
test_p=preprocession(test)

In [ ]:
test_p

In [ ]:
X_test=test_p.drop('id', axis=1)
X_test_d = xgb.DMatrix(X_test)

In [ ]:
X_test

In [ ]:
predictions = bst.predict(X_test_d, iteration_range=(0, bst.best_iteration))

In [ ]:
predictions

In [ ]:
submission = pd.DataFrame(predictions ,columns=label_encoder.inverse_transform([0,1,2]))
submission['id'] = test.sort_values(by='id').reset_index()['id']
cols=['id','home','away','draw']
submission=submission[cols]
display(submission)

In [ ]:
submission.to_csv('submission.csv', index=False)